In [8]:
import pandas as pd
import random
from faker import Faker
from collections import Counter
import os

# 1. INSTALAR LIBRERÍA
# pip install tokenizers
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import random

from sklearn.model_selection import train_test_split
from collections import Counter


In [36]:
# Inicializar Faker para generar datos en español
fake = Faker('es_MX')


# Listas de conceptos clave, ahora más extensas
keywords_retiro = [
    "PAGO", "COMPRA", "COMISION", "RETIRO", "CARGO", "CARGO POR " , "TRANSFERENCIA A", 
    "CHEQUE PAGADO", "IVA", "SPEI ENVIADO", "PAGO CUENTA DE TERCERO", 
    "DOMICILIACION", "CARGO AUTOMATICO", "TRASPASO A CUENTA" , " RETIRO SIN" , "DISPOCISION DE" , 
    "PAGO INTERBANCARIO" , 
]

keywords_deposito = [
    "DEPOSITO", "ABONO", "NOMINA", "SPEI RECIBIDO", "TRANSFERENCIA DE", 
    "PAGO DE", "REEMBOLSO", "TRASPASO DE CUENTA", "A SU FAVOR", "PAGO RECIBIDO" , 
    "DEVOLUCION", "ABONO INVERSION", "PAGO INTERESES" , "TRASPASO ENTRE CUENTAS" , " TRASP ENTRE CUENTAS" , "ABONO POR DEVOLUCION" , 
    "TRANS ENTRE CTAS" , "DISPOCICION DE EFECTIVO"
]

# Listas de entidades específicas para más realismo
bancos = ["BANAMEX", "BBVA", "SANTANDER", "SCOTIABANK", "HSBC", "BANORTE", "STP", "BANCOPPEL", "AZTECA" , "NU BANK" , 
          "BANREGIO" , "BANJERCITO" , "INBURSA" , "BANK OF AMERICA" , "NAFIN" , "BANK OF CHINA" , "ZURICH"
          ] 

merchants = ["AMAZON MX", "MERCADOLIBRE", "NETFLIX", "SPOTIFY", "UBER EATS", "WALMART", "LIVERPOOL", "STEAM" , "OXXO" , 
             "BODEGA AURRERA" , "SEARS" , "AUTOZONE" , "SCOTIACARD" , "DIDI" , "PIZZA HOT"
             
             ]

payment_processors = ["ADYENMX", "MP ECOMMERCE", "PAYPAL", "CONEKTA", "STRIPE"]

tipos_inversion = ["PAGARE", "FONDO DE INVERSION", "CETES", "ACCIONES", "PRLV" , "FINTUAL" , "INVERSION"]


# Modificamos la función para que sea más fácil inyectar ruido
def generar_transaccion_sintetica_ruidosa():
    # --- Generación de la descripción base (lógica similar a la anterior) ---
    if random.random() > 0.45:
        tipo = 'retiro'
        plantillas = [
            "PAGO SERVICIOS {empresa} REF#{ref_num}", "COMPRA EN {merchant}", "{processor}*{merchant}",
            "TRANSFERENCIA A {persona}", " SWEB TRANSFERENCIA A {persona}" ,"SPEI ENVIADO {banco}", "PAGO CUENTA TERCERO",
            "TRASPASO A CUENTA CLABE {clabe}", "INVERSION EN {tipo_inversion}", "RETIRO CAJERO {banco}"
        ]
        # ... (lógica de format para retiros)
        descripcion = random.choice(plantillas).format(
            empresa=fake.company(), merchant=random.choice(merchants), processor=random.choice(payment_processors),
            persona=fake.name(), banco=random.choice(bancos), clabe=fake.numerify('##################'),
            tipo_inversion=random.choice(tipos_inversion), ref=fake.bothify(text='??-####'),
            ref_num=fake.random_number(digits=8)
        )
    else:
        tipo = 'deposito'
        plantillas = [
            "DEPOSITO DE {persona}", "ABONO NOMINA {empresa}", "TRANSFERENCIA SPEI A SU FAVOR",
            "SPEI RECIBIDO DE {banco}", "TRANSFERENCIA DE", "PAGO DE {empresa}", "REEMBOLSO {merchant}",
            "ABONO TRASPASO DE CUENTA", "VENCIMIENTO INVERSION {tipo_inversion}", "DEPOSITO EN EFECTIVO" , "TRASPASO ENTRE CUENTAS"
        ]
        # ... (lógica de format para depósitos)
        descripcion = random.choice(plantillas).format(
            persona=fake.name(), empresa=fake.company(), banco=random.choice(bancos),
            merchant=random.choice(merchants), tipo_inversion=random.choice(tipos_inversion)
        )
    
    descripcion = descripcion.lower()

    # --- INYECCIÓN DE RUIDO (La parte nueva y clave) ---
    # Aplicar ruido a un 60% de las muestras
    if random.random() < 0.6:
        palabras = descripcion.split()
        
        # 1. Pegar palabras aleatoriamente
        if len(palabras) > 1 and random.random() < 0.5:
            idx = random.randint(0, len(palabras) - 2)
            palabras[idx] = palabras[idx] + palabras[idx+1]
            del palabras[idx+1]
        
        # 2. Añadir códigos numéricos al inicio o final
        if random.random() < 0.4:
            palabras.insert(0, fake.numerify('########'))
        if random.random() < 0.4:
            palabras.append(fake.numerify('############'))
            
        # 3. Añadir un monto en medio o al final
        if random.random() < 0.3:
            monto = f"{random.randint(1, 50)},{random.randint(100, 999):03d}.{random.randint(0,99):02d}"
            pos = random.randint(1, len(palabras))
            palabras.insert(pos, monto)
            
        descripcion = " ".join(palabras)

    # El resto de la función es igual
    case_style = random.choice(['original', 'upper'])
    if case_style == 'upper':
        return descripcion.upper(), tipo
    else:
        return descripcion, tipo

In [37]:
import re

def preprocess_text(text):
    """
    Normaliza el texto para el modelo:
    1. Convierte a minúsculas.
    2. Reemplaza cualquier número (entero, decimal, con comas) por un token especial <NUM>.
    3. Elimina espacios extra.
    """
    text = text.lower()
    # Regex para encontrar cualquier secuencia de dígitos, que puede incluir comas y puntos.
    text = re.sub(r'\d[\d,.]*\d|\d', '<NUM>', text)
    # Reemplazar múltiples espacios con uno solo
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [42]:
# --- Parte 2: Función de Pre-procesamiento ---
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d[\d,.]*\d|\d', '<NUM>', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Generar y pre-procesar datos
num_muestras = 30000
df = pd.DataFrame([generar_transaccion_sintetica_ruidosa() for _ in range(num_muestras)], columns=['Descripcion', 'Tipo'])
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['Tipo'])

In [ ]:
df.to_csv(r'', index=False)

In [43]:
train_df.sample(10)

,Descripcion,Tipo
15445,sweb transferencia a natividad orozco munguía,retiro
18924,ABONO TRASPASO DE CUENTA,deposito
24803,traspaso a cuenta clabe 075166089621024600,retiro
16237,TRASPASO A CUENTA CLABE 568953275297774442 441...,retiro
22328,"pago de despacho benítez, trejo y rubio",deposito
9800,vencimiento inversion prlv,deposito
13259,spei enviado stp,retiro
6859,deposito en efectivo,deposito
2539,TRASPASO A CUENTA CLABE865816890904670793 0829...,retiro
4573,"PAGO SERVICIOS CORPORACIN GALLARDO, ÁVALOS Y C...",retiro


In [17]:
# --- Parte 3: Tokenizador entrenado con datos pre-procesados ---
TOKENIZER_FILE = "noisy_transaction_tokenizer.json"
if not os.path.exists(TOKENIZER_FILE):
    print("Entrenando un nuevo tokenizador en datos ruidosos y pre-procesados...")
    tokenizer = Tokenizer(BPE(unk_token="<unk>"))
    tokenizer.pre_tokenizer = Whitespace()
    # ¡IMPORTANTE! El tokenizador debe aprender el token <NUM>
    trainer = BpeTrainer(special_tokens=["<unk>", "<pad>", "<NUM>"], vocab_size=2000)
    # ¡IMPORTANTE! Entrenamos el tokenizador sobre el texto ya pre-procesado
    train_iterator = (preprocess_text(text) for text in train_df['Descripcion'])
    tokenizer.train_from_iterator(train_iterator, trainer=trainer)
    tokenizer.save(TOKENIZER_FILE)
else:
    print("Cargando tokenizador existente.")
tokenizer = Tokenizer.from_file(TOKENIZER_FILE)

Cargando tokenizador existente.


In [18]:
# --- PRUEBA del pipeline de procesamiento ---
print("\n--- Probando el pipeline de procesamiento completo ---")
original_phrase = 'Pago CUENTAde tercero'
preprocessed_phrase = preprocess_text(original_phrase)
encoding = tokenizer.encode(preprocessed_phrase)
print(f"Original: '{original_phrase}'")
print(f"Pre-procesado: '{preprocessed_phrase}'")
print(f"Tokens: {encoding.tokens}")



--- Probando el pipeline de procesamiento completo ---
Original: 'Pago CUENTAde tercero'
Pre-procesado: 'pago cuentade tercero'
Tokens: ['pago', 'cuenta', 'de', 'tercero']


In [19]:
# Esta es la versión CORRECTA y ACTUALIZADA
class NoisyTransactionDataset(Dataset):
    # 1. Ahora recibe la función "preprocessor" en el constructor
    def __init__(self, dataframe, tokenizer, label_map, preprocessor):
        self.df = dataframe
        self.tokenizer = tokenizer
        self.label_map = label_map
        self.preprocessor = preprocessor # La guardamos

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # 2. Toma el texto crudo
        text = self.df.iloc[idx]['Descripcion']
        label_str = self.df.iloc[idx]['Tipo']
        label = self.label_map[label_str]
        
        # 3. ¡PASO CLAVE! Aplica la limpieza ANTES de tokenizar
        preprocessed_text = self.preprocessor(text)
        
        # 4. Tokeniza el texto ya limpio y normalizado
        token_ids = self.tokenizer.encode(preprocessed_text).ids
        
        return torch.tensor(token_ids), torch.tensor(label)

In [20]:
label_map = {'retiro': 0, 'deposito': 1}

In [21]:
def collate_batch(batch):
    label_list, text_list = [], []
    for (_text, _label) in batch:
        label_list.append(_label)
        text_list.append(_text)
    pad_token_id = tokenizer.token_to_id("<pad>")
    text_padded = pad_sequence(text_list, batch_first=True, padding_value=pad_token_id)
    return text_padded, torch.tensor(label_list, dtype=torch.long)

# 4.3. Instanciación del Dataset y DataLoader
train_dataset = NoisyTransactionDataset(train_df, tokenizer, label_map, preprocess_text)
test_dataset = NoisyTransactionDataset(test_df, tokenizer, label_map, preprocess_text)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_batch)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_batch)

In [22]:
class BiLSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=2, 
                            bidirectional=True, batch_first=True, dropout=0.3)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(0.5)

    def forward(self, text):
        embedded = self.dropout(self.embedding(text))
        outputs, (hidden, cell) = self.lstm(embedded)
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        return self.fc(hidden)

# 4.5. Instanciación del modelo y componentes de entrenamiento
VOCAB_SIZE = tokenizer.get_vocab_size()
EMBEDDING_DIM = 100
HIDDEN_DIM = 128
OUTPUT_DIM = 2
PAD_IDX = tokenizer.token_to_id("<pad>")

In [23]:
model = BiLSTMClassifier(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, PAD_IDX)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# --- Parte 5: Bucle de Entrenamiento (formato multi-línea) ---
print("\n--- Iniciando Entrenamiento---")
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for texts, labels in train_loader:
        optimizer.zero_grad()
        predictions = model(texts)
        loss = criterion(predictions, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch+1:02}, Loss: {total_loss/len(train_loader):.4f}')


--- Iniciando Entrenamiento Final ---
Epoch 01, Loss: 0.1135
Epoch 02, Loss: 0.0124
Epoch 03, Loss: 0.0076
Epoch 04, Loss: 0.0075
Epoch 05, Loss: 0.0024


In [24]:
def predict_transaction_final(text, modelo, tokenizer, preprocessor, device='cpu'):
    """
    Función completa para predecir una sola transacción de texto.
    """
    modelo.eval()
    modelo.to(device)
    
    # Aplicar el mismo pre-procesamiento que en el entrenamiento
    preprocessed_text = preprocessor(text)
    
    # Usar el tokenizador para procesar el texto de entrada
    token_ids = tokenizer.encode(preprocessed_text).ids
    
    # Convertir a tensor y añadir una dimensión de "batch" (tamaño 1)
    tensor = torch.LongTensor(token_ids).unsqueeze(0).to(device)
    
    # Realizar la predicción
    prediction = modelo(tensor)
    
    # Obtener el índice de la clase con mayor probabilidad
    _, predicted_idx = torch.max(prediction.data, 1)
    
    # Devolver el nombre de la clase
    inv_label_map = {v: k for k, v in label_map.items()}
    return inv_label_map[predicted_idx.item()]

In [25]:
textos_de_prueba = [
    "spei recibidobanregio 14,5600 245345234523454",
    "Pago cuenta de tercero 3,016.00 00080009000 " , 
    " abr retiro cajero automatico sb $3,000.00 $25,00.00 scotiabank san isidro mx ref 132413243 123412341234"

]

In [26]:
for texto in textos_de_prueba:
    # Llama a la función de predicción
    prediccion = predict_transaction_final(texto, model, tokenizer, preprocess_text)
    
    # Imprime el resultado de forma clara
    print(f"Texto Original: '{texto}'")
    print(f"   ↳ Predicción: {prediccion.upper()}\n")

Texto Original: 'spei recibidobanregio 14,5600 245345234523454'
   ↳ Predicción: DEPOSITO

Texto Original: 'Pago cuenta de tercero 3,016.00 00080009000 '
   ↳ Predicción: DEPOSITO

Texto Original: ' abr retiro cajero automatico sb $3,000.00 $25,00.00 scotiabank san isidro mx ref 132413243 123412341234'
   ↳ Predicción: RETIRO

